In [1]:

# import matplotlib
# matplotlib.use('TkAgg')
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import os
import glob
import fnmatch
from collections import namedtuple, OrderedDict
import scipy.io as sio
from scipy import interpolate, signal
from pyproj import Proj,transform
import sys
sys.path.append('/ocean/ssahu/CANYONS/wcvi/grid/')
from grid_alignment import calculate_initial_compass_bearing as cibc
from bathy_common import *
from matplotlib import path
import xarray as xr
import pandas as pd
import scipy.io as sio
import matplotlib.cm as cm
import cmocean as cmo
import matplotlib.gridspec as gridspec
from dateutil.parser import parse
from salishsea_tools import geo_tools, viz_tools, tidetools, nc_tools
import gsw

/home/ssahu/anaconda3/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
path_to_save ='/data/ssahu/NEP36_Extracted_Months/' #'/home/ssahu/saurav/'


bathy = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/Bathymetry_EastCoast_NEMO_R036_GEBCO_corr_v14.nc')

Z = bathy.variables['Bathymetry'][:]

zlevels = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/CDF_COMB_COMPRESSED/NEP036-N30_IN_20140915_00001440_grid_T.nc').variables['deptht']

lon = bathy['nav_lon'][...]
lat = bathy['nav_lat'][...]

z0 = np.ma.masked_values(Z, 0)

y_wcvi_slice = np.array(np.arange(180,350))
x_wcvi_slice = np.array(np.arange(480,650))

In [3]:
T_2013_file = nc.Dataset('/data/ssahu/NEP36_2013_summer_hindcast/cut_NEP36-S29_1d_20130429_20131025_grid_T_20130429-20130508.nc')

In [4]:
lon_corner = T_2013_file.variables['nav_lon'][0,0]
lat_corner = T_2013_file.variables['nav_lat'][0,0]

In [7]:
print(lon_corner, lat_corner)

-128.857 47.3432


In [8]:
j, i = geo_tools.find_closest_model_point(lon_corner,lat_corner,\
                                          lon,lat,grid='NEMO',tols=\
                                          {'NEMO': {'tol_lon': 0.1, 'tol_lat': 0.1},\
                                           'GEM2.5': {'tol_lon': 0.1, 'tol_lat': 0.1}}) 

print(j,i)

179 479


In [9]:
def tem_sal_timeseries_at_WCVI_locations(grid_scalar):#, j, i):

    temp = grid_scalar.variables['temp'][:, :, 1:, 1:]
    sal =  grid_scalar.variables['salt'][:, :, 1:, 1:]
    
    scalar_ts = namedtuple('scalar_ts', 'temp, sal')

    return scalar_ts(temp, sal)

In [11]:
scalar_ts = tem_sal_timeseries_at_WCVI_locations(T_2013_file)

In [14]:
scalar_ts[1].shape

(10, 32, 170, 170)

In [18]:
temp = np.empty((180,32,y_wcvi_slice.shape[0],x_wcvi_slice.shape[0]))
sal = np.empty((180,32,y_wcvi_slice.shape[0],x_wcvi_slice.shape[0]))


i = 0
for file in sorted(glob.glob('/data/ssahu/NEP36_2013_summer_hindcast/cut_NEP36-S29_1d_*_grid_T_*.nc')):

    scalar_ts = tem_sal_timeseries_at_WCVI_locations(nc.Dataset(file))
    temp[i:i+10,...] = scalar_ts[0]
    sal[i:i+10,...] = scalar_ts[1]
    i = i+10


In [ ]:
SA_loc = np.empty_like(sal)
CT_loc = np.empty_like(sal)
spic = np.empty_like(sal)
rho = np.empty_like(sal)

lat = T_2013_file.variables['nav_lat'][1:,1:]
lon = T_2013_file.variables['nav_lon'][1:,1:]

pressure_loc = gsw.p_from_z(-zlevels[:],np.mean(lat))

for t in np.arange(sal.shape[0]):
    for k in np.arange(sal.shape[1]):
        for j in np.arange(sal.shape[2]):
            for i in np.arange(sal.shape[3]):
                SA_loc[t,k,j,i] = gsw.SA_from_SP(sal[t,k,j,i], pressure_loc[k], lon[j,i], lat[j,i])
                CT_loc[t,k,j,i] = gsw.CT_from_pt(sal[t,k,j,i], temp[t,k,j,i])
                spic[t,k,j,i] = gsw.spiciness0(SA_loc[t,k,j,i],CT_loc[t,k,j,i])
#                rho_jun[t,k,j,i] = gsw.density.rho(SA_loc_jun[t,k,j,i], CT_loc_jun[t,k,j,i], pressure_loc[k])  
                rho[t,k,j,i] = gsw.density.rho(SA_loc[t,k,j,i], CT_loc[t,k,j,i], 0)


In [17]:
y_wcvi_slice.shape[0]

170